In [ ]:
#Checking directory 
%pwd

In [ ]:
import os, sys
import numpy as np
#reference to imp directories
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/statefarm/'

In [ ]:
import importlib
import utils; importlib.reload(utils)
from utils import *


%matplotlib inline

In [ ]:
import vgg16bn; importlib.reload(vgg16bn)
from vgg16bn import Vgg16BN
vgg = Vgg16BN()

In [ ]:
batch_size=64

In [ ]:
%cd $DATA_HOME_DIR
path = DATA_HOME_DIR+'/'

## Creating directory structure

In [ ]:
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

## Creating Validation set based on drivers. 
Need to have separate drivers in validation and train set to prevent overfitting based on drivers. Drivers aren't important but their actions are.

In [ ]:
for i in range(0,10):
    os.mkdir(valid_path+'c'+str(i))

In [ ]:
drivers = df.groupby('subject').groups.keys()
print (drivers)

In [ ]:
import pandas as pd
df= pd.read_csv('driver_imgs_list.csv')
drivers = df.groupby('subject').groups.keys()
print (drivers)

In [ ]:
driver = df.loc[df['subject'] == 'p049']
for index, row in driver.iterrows():
    location = row[1]+'/'
    name = row[2]
    fname = train_path+location+name
    if (os.path.isfile(fname)):
        os.rename(train_path+location+name, valid_path+location+name)

## Setting folder locations

In [ ]:
%cd $DATA_HOME_DIR/

In [ ]:
from shutil import copyfile
test_path = path + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + 'train/'
valid_path=path + 'valid/'

## Moving data to appropriate directories

In [ ]:
%cd $DATA_HOME_DIR/train

In [ ]:
from glob import glob
paths = glob('*/')
print (paths)

Making validation set and all other sets. Switch between copyfile and os.rename as appropriate

In [ ]:
for i in paths:
    tpath = train_path+i
    npath = sample_path+i
    %mkdir $npath
    %cd $tpath
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(1000): copyfile(shuf[i], npath+shuf[i])


Creating sample test

In [ ]:
%cd $fulltest
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(30): copyfile(shuf[i], test_path+shuf[i])

In [ ]:
# Create single 'unknown' class for test set
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

## 2Conv model

In [ ]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, steps_per_epoch=int(np.ceil(batches.samples/64)), nb_epoch=1, validation_data=val_batches, 
                     validation_steps=int(np.ceil(val_batches.samples/64)))
    #model.optimizer.lr = 0.001
    #model.fit_generator(batches, steps_per_epoch=int(np.ceil(batches.samples/64)), nb_epoch=1, validation_data=val_batches, 
     #                validation_steps=int(np.ceil(val_batches.samples/64)))
    return model

In [ ]:
model = conv1(batches)

In [ ]:
def test(self, path, batch_size=8):
        print(path)
        test_batches = get_batches(self, path, shuffle=False, batch_size=batch_size, class_mode=None)
        print (test_batches.samples)
        return test_batches, self.model.predict_generator(test_batches, int(np.ceil(test_batches.samples/batch_size)))
def get_batches(self, path, gen=image.ImageDataGenerator(), shuffle=False, batch_size=8, class_mode='categorical'):
        return gen.flow_from_directory(path, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [ ]:
batches, preds = test(bn_model, test_path, batch_size = batch_size*2)

In [ ]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

In [ ]:
save_array(results_path+'test_preds.dat', preds)
save_array(results_path+'filenames.dat', batches.filenames)

## VGG model

In [ ]:
batches = get_batches(train_path, batch_size=batch_size)
val_batches = get_batches(valid_path, batch_size=batch_size)
test_batches = get_batches(test_path, batch_size=batch_size)

In [ ]:
model = vgg.model

For Vgg16 without BatchNormalization

In [ ]:
#model = vgg.model
# Here, [-1] means that, fetch the last element from the array returned by the for loop
#last_conv_idx = [i for i, l in enumerate(model.layers) if type(l) is Convolution2D][-1]
#conv_layers = model.layers[:last_conv_idx+1]
#conv_model = Sequential(conv_layers)

In [ ]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

In [ ]:
%cd $results_path
rm -R conv*.dat
%cd $path

In [ ]:
#conv_feat2 = model.predict_generator(batches, np.ceil(batches.samples/batch_size))
#conv_val_feat2 = model.predict_generator(val_batches, np.ceil(val_batches.samples/batch_size))
#conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

PreCalculating convolution features for train, val and test data to speed up computations. The precalculated features are stored in bcolz array [due to shortage of RAM] and written to disk to free up RAM. 

In [ ]:
fname = results_path+'conv_featFinalxx.dat'
for i in range(batches.n // batch_size+1):
    conv_feat = model.predict_on_batch(batches.next()[0])
    if not i:
        c = bcolz.carray(conv_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_feat)
    c.shape
c.flush()

In [ ]:
fname = results_path+'conv_val_featFinalxx.dat'
for i in range(val_batches.n // batch_size+1):
    conv_val_feat = model.predict_on_batch(val_batches.next()[0])
    if not i:
        c = bcolz.carray(conv_val_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_val_feat)
c.flush()

In [ ]:
fname = results_path+'conv_test_featFinalxx.dat'
for i in range(test_batches.n // batch_size+1):
    conv_test_feat = conv_model.predict_on_batch(test_batches.next()[0])
    if not i:
        c = bcolz.carray(conv_test_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_test_feat)
c.flush()

In [ ]:
#conv_val_feat = load_array(path+'results/conv_val_feat.dat')
#conv_val_feat2 = load_array(path+'results/conv_val_y.dat')

In [ ]:
fname = results_path+'conv_val_yFinalxx.dat'
save_array(fname, val_labels)
fname = results_path+'conv_train_yFinalxx.dat'
save_array(fname, trn_labels)

Post pre-calculation

In [ ]:
X=bcolz.open(results_path+'conv_featFinalxx.dat', mode='r')
y= bcolz.open(results_path+'conv_train_yFinalxx.dat', mode='r')

In [ ]:
valX=bcolz.open(results_path+'conv_val_featFinalxx.dat', mode='r')
valy= bcolz.open(results_path+'conv_val_yFinalxx.dat', mode='r')

In [ ]:
trn_batches=BcolzArrayIterator(X,y, batch_size=batch_size, shuffle=True)
vl_batches=BcolzArrayIterator(valX,valy, batch_size=batch_size, shuffle=True)

In [ ]:
testX=bcolz.open(path+'results/conv_test_featFinal.dat', mode='r')

In [ ]:
tst_batches=BcolzArrayIterator(testX, batch_size=batch_size, shuffle=False)

In [ ]:
preds = bn_model.predict(conv_val_feat, batch_size=batch_size*2)

## Dense layers on pretrained conv layers
Edit: VGG with Batchnorm using batchnorm-vgg trained model and weights. http://files.fast.ai/models/

Train: 18509
Validate: 3915
Test:  79726

In [ ]:
trn_batches.N

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=model.output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.8

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat2, trn_labels, batch_size=batch_size, nb_epoch=30, 
             validation_data=(conv_val_feat2, val_labels))

In [ ]:
bn_model.fit_generator(trn_batches, nb_epoch=10, steps_per_epoch=np.ceil(trn_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

In [ ]:
bn_model.optimizer.lr=0.01
bn_model.fit_generator(trn_batches, nb_epoch=10, steps_per_epoch=np.ceil(trn_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

In [ ]:
bn_model.save_weights(path+'models/conv8_1Final.h5')

## Pseudo Labelling

In [ ]:
test_pseudo = bn_model.predict_generator(tst_batches, steps=np.ceil(tst_batches.N/batch_size), verbose=1)

In [ ]:
np.set_printoptions(suppress=True)
test_pseudo[3]

In [ ]:
comb_pseudo = np.concatenate([trn_labels, test_pseudo])

In [ ]:
X1=bcolz.open(path+'results/conv_featFinal.dat', mode='r')
trn_batches=BcolzArrayIterator(X1, batch_size=batch_size, shuffle=False)
trn_batches.N

In [ ]:
X2=bcolz.open(path+'results/conv_test_featFinal.dat', mode='r')
tst_batches=BcolzArrayIterator(X2, batch_size=batch_size, shuffle=False)
tst_batches.N

In [ ]:
fname = path+'results/conv_feat_extFinal.dat'
batch = trn_batches.next()
c = bcolz.carray(batch, rootdir=fname, mode='w')
for i in range(trn_batches.N // batch_size):
    batch = trn_batches.next()
    c.append(batch)
for i in range(tst_batches.N // batch_size+1):
    batch = tst_batches.next()
    c.append(batch)
c.flush()

In [ ]:
X3=bcolz.open(path+'results/conv_feat_extFinal.dat', mode='r')

X3.shape

In [ ]:
fname = path+'results/conv_ptest_yFinal.dat'
save_array(fname, comb_pseudo)

In [ ]:
comb_pseudo.shape

Load pre calculated features and train on train+test set

In [ ]:
pX=bcolz.open(path+'results/conv_feat_extFinal.dat', mode='r')
py= bcolz.open(path+'results/conv_ptest_yFinal.dat', mode='r')

In [ ]:
tstComb_batches=BcolzArrayIterator(pX, py, batch_size=batch_size, shuffle=True)

In [ ]:
bn_model.load_weights(path+'models/conv8_1Final.h5')

In [ ]:
bn_model.optimizer.lr=0.001
bn_model.fit_generator(tstComb_batches, nb_epoch=10, steps_per_epoch=np.ceil(tstComb_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

In [ ]:
bn_model.optimizer.lr=0.01
bn_model.fit_generator(tstComb_batches, nb_epoch=10, steps_per_epoch=np.ceil(tstComb_batches.N/batch_size), validation_data=vl_batches, 
                     validation_steps=int(np.ceil(vl_batches.N/batch_size)))

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(preds, 0.9050)).eval()

In [ ]:
pX=bcolz.open(path+'results/conv_test_feat.dat', mode='r')
tst_batches=BcolzArrayIterator(pX, batch_size=batch_size, shuffle=False)

In [ ]:
test_pred = bn_model.predict_generator(tst_batches, steps=np.ceil(tst_batches.N/batch_size), verbose=1)

In [ ]:
subm = do_clip(test_pred,0.8840)

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
subm_name = path+'results/subm2.gz'

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)

## Making submissions, old method

In [ ]:
filenames = val_batches.filenames
expected_labels = val_batches.classes #0 or 1

#Round our predictions to 0/1 to generate labels
print (filenames[1], probs[1])

In [ ]:
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [ ]:
batches = vgg.get_batches(test_path, batch_size=batch_size)

In [ ]:
import re
filenames = batches.filenames
a = []

for idx, val in enumerate(filenames):
    a.append(val[8:])
    

In [ ]:
classes = sorted(val_batches.class_indices, key=val_batches.class_indices.get)
submission = pd.DataFrame(preds, columns=classes)

In [ ]:
print (submission)
submission['img']=a

In [ ]:
cols = submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
submission= submission[cols]

In [ ]:
submission.to_csv('firstSubVGG', index=False)


In [ ]:
submission.shape

In [ ]:
preds = load_array('/home/hearth/ML/course/courses/deeplearning1/nbs/data/dogscats/' + 'test_preds.dat')
filenames = load_array('/home/hearth/ML/course/courses/deeplearning1/nbs/data/dogscats/' + 'filenames.dat')